<a href="https://colab.research.google.com/github/laoktaviana/AI-astar/blob/main/A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pds
import random
import copy
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras import backend as K

In [ ]:
class Maze(object):
    def __init__(self, size=10, blocks_rate=0.1):
        self.size = size if size > 3 else 10
        self.blocks = int((size ** 2) * blocks_rate) 
        self.s_list = []
        self.maze_list = []
        self.e_list = []

    def create_mid_lines(self, k):
        if k == 0: self.maze_list.append(self.s_list)
        elif k == self.size - 1: self.maze_list.append(self.e_list)
        else:
            tmp_list = []
            for l in range(0,self.size):
                if l == 0: tmp_list.extend("#")
                elif l == self.size-1: tmp_list.extend("#")
                else:
                    a = random.randint(-1, 0)
                    tmp_list.extend([a])
            self.maze_list.append(tmp_list)

    def insert_blocks(self, k, s_r, e_r):
        b_y = random.randint(1, self.size-2)
        b_x = random.randint(1, self.size-2)
        if [b_y, b_x] == [1, s_r] or [b_y, b_x] == [self.size - 2, e_r]: k = k-1
        else: self.maze_list[b_y][b_x] = "#"
            
    def generate_maze(self): 
        s_r = random.randint(1, (self.size / 2) - 1)
        for i in range(0, self.size):
            if i == s_r: self.s_list.extend("S")
            else: self.s_list.extend("#")
        start_point = [0, s_r]

        e_r = random.randint((self.size / 2) + 1, self.size - 2)
        for j in range(0, self.size):
            if j == e_r: self.e_list.extend([50])
            else: self.e_list.extend("#")
        goal_point = [self.size - 1, e_r]

        for k in range(0, self.size):
            self.create_mid_lines(k)
        
        for k in range(self.blocks):
            self.insert_blocks(k, s_r, e_r)

        return self.maze_list, start_point, goal_point

In [ ]:
class Field(object):
    def __init__(self, maze, start_point, goal_point):
        self.maze = maze
        self.start_point = start_point
        self.goal_point = goal_point
        self.movable_vec = [[1,0],[-1,0],[0,1],[0,-1]]

    def display(self, point=None):
        field_data = copy.deepcopy(self.maze)
        if not point is None:
                y, x = point
                field_data[y][x] = "@@"
        else:
                point = ""
        for line in field_data:
                print ("\t" + "%3s " * len(line) % tuple(line))

    def get_actions(self, state):
        movables = []
        if state == self.start_point:
            y = state[0] + 1
            x = state[1]
            a = [[y, x]]
            return a
        else:
            for v in self.movable_vec:
                y = state[0] + v[0]
                x = state[1] + v[1]
                if not(0 < x < len(self.maze) and
                       0 <= y <= len(self.maze) - 1 and
                       maze[y][x] != "#" and
                       maze[y][x] != "S"):
                    continue
                movables.append([y,x])
            if len(movables) != 0:
                return movables
            else:
                return None

    def get_val(self, state):
        y, x = state
        if state == self.start_point: return 0, False
        else:
            v = float(self.maze[y][x])
            if state == self.goal_point: 
                return v, True
            else: 
                return v, False

In [ ]:
size = 10
barriar_rate = 0.1

maze_1 = Maze(size, barriar_rate)
maze, start_point, goal_point = maze_1.generate_maze()
maze_field = Field(maze, start_point, goal_point)

maze_field.display()

	  #   #   #   #   S   #   #   #   #   # 
	  #   0   #   0   0   0  -1   #  -1   # 
	  #   0  -1  -1   0  -1  -1   0   #   # 
	  #   0   0   0  -1   0   #  -1  -1   # 
	  #  -1  -1   0   0  -1   0   0   #   # 
	  #  -1  -1  -1  -1  -1   0   0   0   # 
	  #  -1  -1   0  -1   #   0   #  -1   # 
	  #  -1   0   0  -1   0   #  -1  -1   # 
	  #  -1   #   0   #   0   0   0  -1   # 
	  #   #   #   #   #   #  50   #   #   # 


In [ ]:
class Node(object):    
    def __init__(self, state, start_point, goal_point):
        self.state = state
        self.start_point = start_point
        self.goal_point = goal_point
        self.hs = (self.state[0] - self.goal_point[0]) ** 2 + (self.state[1] - self.goal_point[1]) ** 2
        self.fs = 0
        self.parent_node = None
    
    def confirm_goal(self):
        if self.goal_point == self.state: return True
        else: return False

In [ ]:
class NodeList(list):
    def find_nodelist(self, state):
        node_list = [t for t in self if t.state==state]
        return node_list[0] if node_list != [] else None
    def remove_from_nodelist(self, node):
        del self[self.index(node)]

In [ ]:
class Aster_Solver(object):
    def __init__(self, maze, start_point, goal_point, display=False):
        self.Field = maze
        self.start_point = start_point
        self.goal_point = goal_point
        self.open_list = NodeList()
        self.close_list = NodeList()
        self.steps = 0
        self.score = 0
        self.display = display
        
    def set_initial_node(self):
        node = Node(self.start_point, self.start_point, self.goal_point)
        node.start_point = self.start_point
        node.goal_point = self.goal_point 
        return node
                
    def go_next(self, next_actions, node):
        node_gs = node.fs - node.hs
        for action in next_actions:
            open_list = self.open_list.find_nodelist(action)
            dist = (node.state[0] - action[0]) ** 2 + (node.state[1] - action[1]) ** 2
            if open_list:
                if open_list.fs > node_gs + open_list.hs + dist:
                    open_list.fs = node_gs + open_list.hs + dist
                    open_list.parent_node = node
            else:
                open_list = self.close_list.find_nodelist(action)
                if open_list:
                    if open_list.fs > node_gs + open_list.hs + dist:
                        open_list.fs = node_gs + open_list.hs + dist
                        open_list.parent_node = node
                        self.open_list.append(open_list)
                        self.close_list.remove_from_nodelist(open_list)
                else:
                    open_list = Node(action, self.start_point, self.goal_point)
                    open_list.fs = node_gs + open_list.hs + dist
                    open_list.parent_node = node
                    self.open_list.append(open_list)
    
    def solve_maze(self):
        node = self.set_initial_node()
        node.fs = node.hs
        self.open_list.append(node)
        
        while True:            
            node = min(self.open_list, key = lambda node:node.fs)
            print ("current state:  {0}".format(node.state))
            
            if self.display:
                self.Field.display(node.state)
            
            reward, tf = self.Field.get_val(node.state)
            self.score =  self.score + reward
            print("current step: {0} \t score: {1} \n".format(self.steps, self.score))
            self.steps += 1
            if tf == True:
                print("Goal!")
                break

            self.open_list.remove_from_nodelist(node)
            self.close_list.append(node)
            
            next_actions = self.Field.get_actions(node.state)   
            self.go_next(next_actions, node)

In [ ]:
astar_Solver = Aster_Solver(maze_field, start_point, goal_point, display=True)
astar_Solver.solve_maze()

current state:  [0, 4]
	  #   #   #   #  @@   #   #   #   #   # 
	  #   0   #   0   0   0  -1   #  -1   # 
	  #   0  -1  -1   0  -1  -1   0   #   # 
	  #   0   0   0  -1   0   #  -1  -1   # 
	  #  -1  -1   0   0  -1   0   0   #   # 
	  #  -1  -1  -1  -1  -1   0   0   0   # 
	  #  -1  -1   0  -1   #   0   #  -1   # 
	  #  -1   0   0  -1   0   #  -1  -1   # 
	  #  -1   #   0   #   0   0   0  -1   # 
	  #   #   #   #   #   #  50   #   #   # 
current step: 0 	 score: 0 

current state:  [1, 4]
	  #   #   #   #   S   #   #   #   #   # 
	  #   0   #   0  @@   0  -1   #  -1   # 
	  #   0  -1  -1   0  -1  -1   0   #   # 
	  #   0   0   0  -1   0   #  -1  -1   # 
	  #  -1  -1   0   0  -1   0   0   #   # 
	  #  -1  -1  -1  -1  -1   0   0   0   # 
	  #  -1  -1   0  -1   #   0   #  -1   # 
	  #  -1   0   0  -1   0   #  -1  -1   # 
	  #  -1   #   0   #   0   0   0  -1   # 
	  #   #   #   #   #   #  50   #   #   # 
current step: 1 	 score: 0.0 

current state:  [2, 4]
	  #   #   #   #   S   #   #   